In [1]:
import numpy as np
import scipy as sp
from scipy import io,integrate,sparse
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

from kpm_bin import *

from IPython.display import clear_output
np.set_printoptions(linewidth=300)
%load_ext autoreload

%autoreload 2

## TeX preamble
preamble = r"""
\usepackage[no-math]{fontspec}
\setmainfont[]{Helvetica Condensed}

\usepackage{amsmath,amssymb,amsthm}
\usepackage{mathastext}
"""

mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': preamble,
})

In [2]:
import kwant
from matplotlib import pyplot

lat = kwant.lattice.general([(0, 0.5, 0.5), (0.5, 0, 0.5), (0.5, 0.5, 0)],
                            [(0, 0, 0), (0.25, 0.25, 0.25)])
a, b = lat.sublattices

def make_cuboid(a=15, b=10, c=5):
    t = 1
    def cuboid_shape(pos):
        x, y, z = pos
        return 0 <= x < a and 0 <= y < b and 0 <= z < c

    syst = kwant.Builder()
    syst[lat.shape(cuboid_shape, (0, 0, 0))] = 0
    syst[lat.neighbors()] = 1

    return syst


# the standard plotting style for 3D is mainly useful for
# checking shapes:
syst = make_cuboid(a=35,b=20,c=10)
fsyst = syst.finalized()

fig,ax = plt.subplots(1,1,figsize=(7,7),subplot_kw={'projection':'3d'})

def family_color(site):
    return plt.cm.magma(.2) if site.family == a else plt.cm.magma(.5)
    
kwant.plot(syst,site_color=family_color,ax=ax,dpi=600);
ax.set_axis_off()
ax.pbaspect = [2.0, 0.6, 0.25]

plt.savefig('imgs/graphene_shape.png',dpi=600)

In [3]:
H = fsyst.hamiltonian_submatrix(sparse=True)
d = H.shape[0]

In [4]:
m = 10
k = 500
np.random.seed(0)

Rvs = []
Rws = []
αβs = []
for _ in range(m):
    print(f'random sample {_}')
    clear_output(wait=True)

    v = np.random.randn(d)
    v /= np.linalg.norm(v)

    (α,β) = lanczos(H,v,k)
    αβs.append((α,β))

    Hk = np.diag(β,1) + np.diag(β,-1) + np.diag(np.append(α,0))

    Rv,Rvec = np.linalg.eigh(Hk[:k,:k])
    Rvs.append(Rv)
    Rws.append(Rvec[0]**2)
    
Rvs = np.array(Rvs)
Rws = np.array(Rws)

random sample 9


In [5]:
%%timeit
sp.sparse.linalg.eigsh(H,k=1,which='LA',tol=0.02,return_eigenvectors=False)

48 ms ± 4.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%%timeit
(α,β) = lanczos(H,v,120)

152 ms ± 24.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
Emin_true = np.min(Rvs,axis=0)[0]
Emax_true = np.max(Rvs,axis=-1)[0]

In [8]:
methods = ['T','Thd','Tspike']

k_eff = min(240,2*k)

ϵs = [1e-1]

ϵ = .015
Emin = Emin_true - ϵ*(Emax_true - Emin_true)
Emax = Emax_true + ϵ*(Emax_true - Emin_true)

EE = np.linspace(Emin,Emax,500)
EE = np.hstack([EE,np.linspace(-1e-2,+1e-2,100)])
EE = np.sort(EE)
    


fig,axs = plt.subplots(1,1,figsize=(6,3))
plt.subplots_adjust(bottom=.18,top=.9)
axs = [axs]

for i,method in enumerate(methods):
    
    for ϵ in ϵs:
        Emin = Emin_true - ϵ*(Emax_true - Emin_true)
        Emax = Emax_true + ϵ*(Emax_true - Emin_true)
    
        μs = []
    
        if method == 'Tspike':
            w = 1e-2
            Esl,Esr = -w,+w
            Er = max(Rvs[Rvs < -w])
            El = min(Rvs[Rvs > +w])
            intervals = np.array([[Esl,Esr],\
                                  [Emin_true - ϵ*(Er - Emin_true),Emax_true + ϵ*(Emax_true - El)]])

            αβ0s = [get_chebT_recurrence(2*k,int[0],int[1]) for int in intervals]

            ws = .05
            wb = (1-ws)
            weights = [ws,wb]
            
            (γ,δ) = get_multiint_op_recurrence(αβ0s,weights,2*k)
            σ = lambda x: ws*get_chebT_density(*intervals[0])(x) + wb*get_chebT_density(*intervals[1])(x)

        elif method =='T' or method == 'Thd':
            (γ,δ) = get_chebT_recurrence(2*k,Emin,Emax)
            σ = get_chebT_density(Emin,Emax)
        
        for (α,β) in αβs:
    
            Hk = np.diag(β,1) + np.diag(β,-1) + np.diag(np.append(α,0))
            e0 = np.zeros(k+1)
            e0[0] = 1
        
            μ = get_moments(Hk,e0,2*k,γ,δ)
    
            μs.append(μ)

        k_eff_loc = k_eff
        k_eff_loc = k_eff
        if method == 'Thd':
            k_eff_loc = 800
            
        g = np.ones(k_eff_loc)
        if method =='T' or method == 'Thd':
            g = jackson_weights(k_eff_loc)

        dρdσ = get_op_expansion(g*np.mean(μs,axis=0)[:k_eff_loc],γ,δ)
        ρ_KPM = lambda E: σ(E)*dρdσ(E)

        
    styles = [{'color':plt.cm.magma(.85),'lw':1,'alpha':1},\
              {'color':plt.cm.magma(.5),'lw':1,'alpha':1},\
              {'color':plt.cm.magma(.12),'lw':1}]

    axs[0].plot(EE,ρ_KPM(EE),**styles[i],label=rf'$\epsilon={ϵ}$')

axs[0].set_ylim(-.01,.3)
axs[0].set_xlabel('energy \emph{E}')
axs[0].set_ylabel('density of states')

def family_colors(site):
    return 0 if site.family == a else 1    

# with PIL.Image.open(r"imgs/graphene_shape.png") as im:
#     axs[1].imshow(im.crop((900,1340,3500,2750)))

# axs[1].set_axis_off()

plt.savefig('imgs/graphene.pdf')

/home/tyler/Documents/Research Code/spectrum_adaptive_kpm/kpm_bin/kpm.py:56: ComplexWarning: Casting complex values to real discards the imaginary part
  μ[i+1] = v.T@q
